# Keras Playground #
This module is designed for 
* Rewritting the Keras Training Kernel To Support Different Configurations 
* Load the Old Weights but with build a new model for the new molecule. 
* Retrain The Keras Model 

In [11]:

import keras 
from keras.layers import Input, Dense, merge
from keras.models import Model, Sequential

from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
import numpy as np

In [71]:
nUnit = 34 #int(os.environ['unit_number'])

import keras.backend as K 
def Nguyen_Widrow_init(shape, dtype=None):
    '''Nguyen-Widrow initialization for weights and biases
    Initialize the weight and biases for each layer.
    
            Args:
                shape: ???? (Guess: Shape of the layer to create)
            
            Output:
                [w,b]
                w: represents an array of weight
                b: represents an array of bias
    
    Comment:
    This is the function provided by Nguyen's original code. 
    Junmian Zhu did the documentation. It is not 100% sure about the 
    algorithm used here. 
    
    '''
    n_input, n_unit = shape[0], shape[1]  # n_input: number of units of prev. layer without bias 
    w_init = np.random.rand(n_input, n_unit) *2 -1
    norm = 0.7 * n_unit ** (1. / n_input)
    # normalize 
    w = norm * w_init/np.sqrt(np.square(w_init).sum(axis=0).reshape(1, n_unit))
    if n_unit>1:
        b = norm * np.linspace(-1,1,n_unit) * np.sign(w[0,:]) 
    else: 
        b = np.zeros((n_unit,)) 
    return [w, b]

def create_base_network(input_dim, name):
    '''3-layer base network to be shared among atoms of the same species 
    '''    
    seq = Sequential()
    seq.add(Dense(nUnit, input_shape=(input_dim,), activation='tanh'))
    shape0 = (seq.layers[0].input_shape[1], seq.layers[0].output_shape[1]) 
    seq.layers[0].set_weights(Nguyen_Widrow_init(shape0))
    
    seq.add(Dense(nUnit, activation='tanh'))
    shape1 = (seq.layers[1].input_shape[1], seq.layers[1].output_shape[1]) 
    seq.layers[1].set_weights(Nguyen_Widrow_init(shape1))
    
    seq.add(Dense(nUnit, activation='tanh'))
    shape2 = (seq.layers[2].input_shape[1], seq.layers[2].output_shape[1]) 
    seq.layers[2].set_weights(Nguyen_Widrow_init(shape2))

    seq.add(Dense(1, activation='linear'))
    shape3 = (seq.layers[3].input_shape[1], seq.layers[3].output_shape[1]) 
    seq.layers[3].set_weights(Nguyen_Widrow_init(shape3)) 
    
    seq.name = name
    return seq

def main_network(input_dim_list, ele_arr, n_symm_func, n_atoms, n_ele):
    '''
    Main neural network with shared sub-structures.
    Currently it supports a sub-network with additional features. 
    '''
    type_nb = len(input_dim_list)
    element_network = {}
    inp = {'element':{}, 'rc':{}}
    
    element_processed = {}  # Will keep the input of the atomic symmetry function
    #rc_processed = []       # Will keep the input of the features, Mulliken Charge of the atom, etc.
    
    # The sub-network that computes E(RC), the correction for geometry
    # dims[2] = number of features in the second input.
    # Assume that it must have hydrogen
    #rc_network = create_base_network(input_dim_list['H'][1], "rc-subnet")   
    # atomic inputs of the same species are processed through the same sub-network.
    
    
#     #for at_type in range(type_nb):
#     for at_type in ['H', 'O', 'N', 'C']:
#         dims = input_dim_list[at_type]
#         # Create a sub-network for each element. 
#         element_network[at_type] = create_base_network(dims[0], at_type + '-subnet')

    for ele_idx in range(n_ele):
        element_network[ele_idx] = create_base_network(n_symm_func, str(ele_idx))

    inp = Input(shape=(None,  None, n_symm_func + 1),dtype='float32') 
    
    
    for at_idx in range(n_atoms):
        ele_idx = inp[at_idx, 0, 0] # Which Represents the element 
        element_processed[at_idx] = element_network[ele_idx](inp[at_idx,:, 1:]) 
    
    outputs = [element_processed[at_idx] for at_idx in range(n_atoms)]
    
    # Define the output (predicted total energy) as a sum of all atomic energies
    main_output = keras.layers.add(outputs)    

    # Organize the input structure by using the structure as the network defined above. 
    #     # because Keras will take multiple input one by one according to the sequence. 
    #     inputs = []
    #     #for at_type in range(type_nb):
    #     for at_type in at_idx_map:    
    #         dims = input_dim_list[at_type]
        
    #         #for atA in range(dims[0]):
    #         for atA in at_idx_map[at_type]:
    #             inputs.append(inp['element'][at_type][atA])
    #             #inputs.append(inp['rc'][at_type][atA])  
    
    # Finalizing building the main network model. 
    main_network = Model(inp, main_output)  

    return main_network

In [72]:
n_symm_func = 276 
#input_dim_list = [(len(at_idx_map[at]) ,nSymFunc, nb_feat) for at in at_idx_map.keys()]

input_dim_list = {}
input_dim_list['H'] = (n_symm_func, 1)
input_dim_list['C'] = (n_symm_func, 1)
input_dim_list['O'] = (n_symm_func, 1)
input_dim_list['N'] = (n_symm_func, 1)


In [73]:
ele_arr = ['H', 'C', 'O', 'N']

In [74]:
model = main_network(input_dim_list, ele_arr,10,  n_symm_func, 4)
model.summary()

TypeError: 'Tensor' object cannot be interpreted as an integer

In [ ]:
model_folder = "model_2B_nUnit"
!mkdir $model_folder

check1 = model_folder +'/' + str(nUnit) + '.hdf5'
checkpointer = ModelCheckpoint(filepath=check1, verbose=0,  monitor='val_mean_squared_error',\
                               mode='min', save_best_only=True)
earlystop = EarlyStopping(monitor='val_mean_squared_error', mode='min', patience=100, verbose=0)

In [ ]:
model.compile(loss='mean_squared_error',
              optimizer=Adam(lr=0.001,decay=1e-5),
              metrics=['mean_squared_error', 'mean_absolute_error'])

history = model.fit(inp_train, y_train, \
                    callbacks=[checkpointer, earlystop],
                    batch_size=64, epochs=1200, shuffle=True,
                    verbose=0, validation_data=(inp_val, y_val)
                    )
plt.plot(history.history['val_mean_squared_error'])
plt.plot(history.history['mean_squared_error'])
plt.show()

In [ ]:
model